In [17]:
# 필요한 라이브러리 import하기

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 

/dshome/WoongLab/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# train data 로드하기
train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_train_data.csv')

In [10]:
# test data 로드하기

test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_test_data.csv')

In [11]:
train_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,27.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,2,2.0,1.0,0.0,0.0,13.0,4,1,0.0
1,1.0,1.0,1.0,25.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,0.0,10.0,5,6,0.0
2,0.0,0.0,1.0,24.0,1.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,7.0,3,6,0.0
3,1.0,1.0,1.0,24.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,2,0.0,30.0,1.0,0.0,11.0,3,3,0.0
4,0.0,0.0,1.0,20.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,1,15.0,0.0,0.0,0.0,2.0,5,5,0.0


In [12]:
test_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,30.0,1.0,1.0,0,0.0,0.0,1.0,...,0.0,1,0.0,0.0,0.0,1.0,11.0,5,7,0.0
1,1.0,1.0,1.0,27.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,1,4.0,0.0,0.0,0.0,11.0,5,5,0.0
2,1.0,0.0,1.0,31.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,2,3.0,3.0,0.0,0.0,6.0,5,6,0.0
3,0.0,1.0,1.0,26.0,1.0,0.0,0,1.0,0.0,0.0,...,0.0,2,0.0,0.0,0.0,0.0,12.0,5,5,0.0
4,1.0,1.0,1.0,25.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,9.0,4,6,0.0


In [13]:
test_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker']

In [14]:
train_X=train_data.loc[:,test_stage_features] 
train_y=train_data['HeartDiseaseorAttack']
new_test_data_X=test_data[test_stage_features]
new_test_data_y=test_data['HeartDiseaseorAttack']

In [15]:
train_y.value_counts()

0.0    183830
1.0     19114
Name: HeartDiseaseorAttack, dtype: int64

In [16]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [18]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-13 04:16:39,631] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_116204/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-13 04:16:50,925] Trial 0 finished with value: 0.6654453912030831 and parameters: {'learning_rate': 0.030985402777562954, 'max_depth': 9, 'n_estimators': 652}. Best is trial 0 with value: 0.6654453912030831.
/tmp/ipykernel_116204/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-13 04:16:53,468] Trial 1 finished 

Best hyperparameters: {'learning_rate': 0.015463591215453126, 'max_depth': 3, 'n_estimators': 990}
Best AUC: 0.6703


In [19]:
lgb=lgb.LGBMClassifier(learning_rate=0.015463591215453126,max_depth=3,n_estimators=990,scale_pos_weight=ratio,random_state=42)

In [20]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.015463591215453126, max_depth=3,
               n_estimators=990, random_state=42,
               scale_pos_weight=9.617557811028565)

In [21]:
prob=lgb.predict_proba(new_test_data_X)[:,1]

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [23]:
get_clf_prob(new_test_data_y,prob)

임계값:  0.5
오차행렬
[[25995 19962]
 [ 1531  3248]]
정확도: 0.5764, 정밀도: 0.1399, 재현율: 0.6796, F1:0.4698, AUC: 0.6665
